# MCU2024-A

# Import Packages

In [209]:
import numpy as np
import pandas as pd

# 问题一

## 基本功能函数

### 单位化向量

In [211]:
def compute_unit_vector(vx, vy):
    """
    计算单位向量
    输入: vx -  x 分量
          vy -  y 分量
    输出: 单位向量 (vx_n, vy_n)
    """
    # 计算速度的大小
    magnitude = np.sqrt(vx**2 + vy**2)
    
    # 计算单位向量
    vx_n = vx / magnitude
    vy_n = vy / magnitude
    
    return (vx_n, vy_n)

### 由 $\theta$ 求解极径r的值
螺线方程为： $r=8.8 - \frac{0.55}{2\pi}\theta $

In [212]:
def compute_radius(θ):
    """
    计算螺线方程 r = 8.8 + (0.55 / (2 * pi)) * theta 的极径 r
    输入: theta - 角度（弧度制）
    输出: 对应的极径 r
    """
    # 螺线方程
    r = 8.8 + (0.55 / (2 * np.pi)) * θ
    return r

### 由 $\theta$ 计算点坐标

In [214]:
def compute_coordinates(θ):
    """
    计算点的坐标 (r cos(θ), r sin(θ))
    输入: theta - 角度（弧度制）
    输出: 对应的坐标 (x, y)
    """
    # 计算极径
    r = compute_radius(θ)
    
    # 计算坐标
    x = r*np.cos(θ)
    y = r*np.sin(θ)
    return (x, y)


### 求解龙头前把手的位置
$\overrightarrow {v_1}=(r_1 cos \theta_1, -r_1 sin \theta_1)$

In [216]:
def compute_hand_position(θ):
    """
    计算在时间 t 时刻龙头前把手的位置
    输入: theta - 角度（弧度制）
    输出: 对应的坐标 (x, y)
    """
    
    return compute_coordinates(θ)

##  求解位置


### 求出龙头在300s内的所有 $\theta$ 值
从0秒到300秒，共301个时间点

#### 计算函数

In [218]:
from scipy.optimize import fsolve

def f(x):
    """
    计算函数 f(x)
    输入: x - 自变量
    输出: f(x) 的值
    """
    return 0.5 * x * np.sqrt(x**2 + 1) + 0.5 * np.log(np.abs(x + np.sqrt(x**2 + 1)))

def g(x, t):
    """
    计算函数 g(x) - t
    输入: x - 自变量
           t - 目标值
    输出: g(x) - t 的值
    """
    term1 = f(x + 32 * np.pi)
    term2 = f(32 * np.pi)
    return (0.55 / (2 * np.pi)) * (term2 - term1) - t

def solve_θ_of_dragonhead(t_value):
    """
    解方程 g(x) = t 的 x 值
    输入: t_value - 目标值 t
    输出: 对应的 x 值
    """
    # 初始猜测值为0
    initial_guess = np.radians(0)

    try:
        # 使用 fsolve 求解 g(x) - t = 0 的方程
        x_value = fsolve(g, x0=initial_guess, args=(t_value))
        return x_value[0]
    except Exception as e:
        print(f"Failed to solve: {e}")
        return None


#### 计算并存储龙头的 θ 值

In [220]:
# 初始化时间数组
time_array = np.arange(301)  # 从 0 到 300 的整数数组

# 计算每个时间点的角度值（弧度制）
theta_head_value = np.array([solve_θ_of_dragonhead(t) for t in time_array])

In [221]:
# 输出300s 内的所有龙的 θ 值
print("前几个 θ 值 (弧度制):")
print(theta_head_value)  


前几个 θ 值 (弧度制):
[  0.          -0.11369503  -0.22751892  -0.34147211  -0.45555504
  -0.56976816  -0.68411191  -0.79858674  -0.91319311  -1.02793147
  -1.14280227  -1.25780598  -1.37294306  -1.48821397  -1.60361918
  -1.71915916  -1.83483439  -1.95064533  -2.06659248  -2.18267631
  -2.2988973   -2.41525595  -2.53175274  -2.64838816  -2.76516272
  -2.88207691  -2.99913123  -3.11632619  -3.2336623   -3.35114006
  -3.46876     -3.58652263  -3.70442847  -3.82247804  -3.94067187
  -4.0590105   -4.17749445  -4.29612426  -4.41490047  -4.53382362
  -4.65289427  -4.77211295  -4.89148024  -5.01099667  -5.13066282
  -5.25047924  -5.37044651  -5.49056519  -5.61083586  -5.7312591
  -5.85183549  -5.97256562  -6.09345007  -6.21448944  -6.33568432
  -6.45703533  -6.57854305  -6.70020811  -6.8220311   -6.94401266
  -7.0661534   -7.18845395  -7.31091492  -7.43353697  -7.55632072
  -7.67926682  -7.8023759   -7.92564863  -8.04908566  -8.17268764
  -8.29645524  -8.42038912  -8.54448996  -8.66875843  -8.79319

### 求出对应于每个 $\theta$ 的 所对应的所有龙身的 $\theta$ 值

#### 计算函数

In [222]:
from scipy.optimize import fsolve

def g_function(θ2, θ1, b):
    """
    计算方程 g(θ2) - b^2 的值，用于求解 θ2
    输入: theta2 - 待求解的角度 θ2（弧度制）
           theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: g(θ2) - a^2 的值
    """
    r1 = compute_radius(θ1)
    r2 = compute_radius(θ2)
    # g(θ2) 的表达式
    g_θ2 = r2**2 - 2 * r1 * r2 * np.cos(θ2 - θ1) + r1**2
    return g_θ2 - b**2

def compute_theta_next(θ1, b):
    """
    使用数值方法计算 θ2，使得 g(θ2) = b^2 且 θ2 - θ1 ∈ (0, 2π)
    输入: theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: 满足条件的 θ2 的值
    """
    # 定义多个初始猜测值，用于 fsolve 寻找不同的解
    initial_guesses = [ θ1+0.2, θ1+0.4, θ1+0.7, θ1+0.9]  
    solutions = []

    # 对每个初始猜测值进行求解
    for guess in initial_guesses:
        θ2_solution = fsolve(g_function, guess, args=(θ1, b))
        θ2 = θ2_solution[0]

        # 检查解是否满足  θ2 - θ1 ∈ (-2π, 0)
        if  0 < (θ2 - θ1) < 2*np.pi:
            solutions.append(θ2)

    # 返回满足条件的最小解
    if solutions:
        unique_solutions = np.unique(solutions)  # 去重
        return min(solutions)
    else:
        return None  # 如果没有满足条件的解，返回 None

#### 计算每个时间点的所有龙身的 θ 值

In [224]:
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建二维数组来存储每个时间点的所有龙身的 θ 值
theta_body_values_of_each_time = np.empty((301, 223))  # 301 行，222 列；301个时间点，221个龙身，1个龙尾

# 计算每个时间点的所有龙身的 θ 值
for t in range(301):
    temp =  theta_head_value[t] # 当前时间内点的龙头对应的θ值
    for i in range(223):
        if i == 0:
            theta_body_values_of_each_time[t, i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time[t, i] # 更新当前时间点的龙身对应的θ值
        else:
            theta_body_values_of_each_time[t, i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time[t, i] # 更新当前时间点的龙身对应的θ值
            

In [225]:
# 定义保存数组到 CSV 文件的函数
def save_array_to_csv(array, filename):
    """
    将二维数组保存到 CSV 文件
    输入:
        array - 要保存的数组
        filename - 保存的文件名
    """
    # 使用 numpy.savetxt 将数组保存到 CSV 文件中
    np.savetxt(filename, array, fmt='%.6f', delimiter=',')  # fmt 控制浮点数精度，delimiter 控制分隔符为逗号

# 调用函数将数组保存到 CSV 文件中
save_array_to_csv(theta_body_values_of_each_time, './Q1/theta_body_values_Q1.csv')

print("二维数组已保存到 ./Q1/theta_body_values_Q1.csv 文件中。")


二维数组已保存到 ./Q1/theta_body_values_Q1.csv 文件中。


In [226]:
# 将所有时刻的所有点的位置信息存储到同一变量中，301行，223列，第一列为龙头，其余为龙身
theta_values_of_each_time = np.hstack((theta_head_value.reshape(-1, 1), theta_body_values_of_each_time))

## 求解速度

### 计算点的速度

In [227]:
def compute_radius_derivative(θ):
    """
    计算 r(θ) = 8.8 - (0.55 / (2π)) * θ 的导数 
    dr/dθ= -0.55 / (2π)
    输出: 导数表达式的值
    """
    
    # 定义 r(theta) 的方程
    dr =   (0.55 / (2 * np.pi)) 
    
    return dr

In [228]:
def compute_velocity(θ):
    """
    计算点的速度
    """
    # 计算r的值
    r = compute_radius(θ)
    # 计算r的导数值
    dr = compute_radius_derivative(θ)

    # 计算点的速度
    vx = - r * np.sin(θ) + dr * np.cos(θ)
    vy =   r * np.cos(θ) + dr * np.sin(θ)
    
    return (vx, vy)

### 板凳的方向向量
$\vec{l}=(r_{n+1}\cos{\theta_{n+1}}-r_n\cos{\theta_n},r_{n+1}\sin{\theta_{n+1}}-r_n\sin(\theta_n))=(\cos{\alpha_n},sin{\alpha_n})$

In [231]:
def compute_bench_direction_vector(θ_n, θ_n1):
    """
    计算板凳的方向向量
    输入: 
        r_n, theta_n - 当前点的极径和角度（弧度制）
        r_n1, theta_n1 - 下一个点的极径和角度（弧度制）
    输出: 
        向量 (lx, ly) 
    """
    # r_n 和 r_n1
    r_n = compute_radius(θ_n)
    r_n1 = compute_radius(θ_n1)

    # 计算方向向量的 x 和 y 分量
    lx = r_n * np.cos(θ_n) - r_n1 * np.cos(θ_n1) 
    ly = r_n * np.sin(θ_n) - r_n1 * np.sin(θ_n1)  
    
    return lx, ly

### 计算并存储t时刻龙头的前把手的速度的大小


In [233]:
#  speed_of_head 表示每个时刻的龙头的速度大小
speed_of_head= 1

### 计算并存储t时刻龙头的所有前把手的速度的大小
$u_n$ 表示速度的大小
$u_{n+1} = \frac{\vec{v_n}\cdot \vec l_n}{\overrightarrow{v_{n+1}} \cdot \vec l_n} \cdot u_n$

#### 计算函数

In [234]:
def compute_velocity_next(θ_n, θ_n1, u_n):
    """
    计算速度大小 u_{n+1}
    
    参数：
    u_n: 当前点速度的大小
    v_n: 当前点的速度方向向量
    v_n1: 下一点的速度方向向量
    l: 参考方向的单位向量
    
    返回：
    u_{n+1}: 下一点的速度大小
    """

    # 计算速度方向向量
    v_n = compute_unit_vector(*compute_velocity(θ_n))
    v_n1 = compute_unit_vector(*compute_velocity(θ_n1))

    # 计算板凳的方向向量
    l_n_x, l_n_y = compute_bench_direction_vector(θ_n, θ_n1)
    l_n = np.array([l_n_x, l_n_y])
    
    # 计算点乘
    numerator = np.dot(v_n, l_n) # 计算分子
    denominator = np.dot(v_n1, l_n)   # 计算分母
    
    # 确保分母不为零
    if denominator == 0:
        raise ValueError("Denominator is zero, cannot divide.")
    
    # 更新速度大小
    u_n1 = (numerator / denominator) * u_n

    return u_n1


In [235]:
# 建立一个空的数组来存储每个时间点龙身每个点的速度
velocity_body_values_of_each_time = np.empty((301, 224))  # 301 行，223 列；301个时间点，一个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

for t in range(301):
    temp = speed_of_head # 计算当前时间点的龙头速度大小
    velocity_body_values_of_each_time[t, 0] = temp # u0 存放龙头的速度大小
    for i in range(0,223):
        velocity_body_values_of_each_time[t, i+1] = compute_velocity_next(theta_values_of_each_time[t,i], theta_values_of_each_time[t,i+1], temp)
        temp = velocity_body_values_of_each_time[t, i+1]

In [236]:
# 将矩阵保存到 txt 文件中
output_file_path = "./Q1/velocity_body_values.txt"  # 文件路径，可以根据需要修改

# 保存矩阵到 txt 文件，格式化为小数点后两位
np.savetxt(output_file_path, velocity_body_values_of_each_time, fmt='%.6f', delimiter=',', header='Velocity values for each time step')

print(f"速度矩阵已成功保存到 {output_file_path}")

速度矩阵已成功保存到 ./Q1/velocity_body_values.txt


## 输出结果整理


### 位置信息

In [237]:
# 将所有时刻的所有点的位置信息存储到同一变量中，301行，223列，第一列为龙头，其余为龙身, 最后一列为龙尾后把手
theta_values_of_each_time = np.hstack((theta_head_value.reshape(-1, 1), theta_body_values_of_each_time))

# 计算每个点的 r 值
r_values = np.vectorize(compute_radius)(theta_values_of_each_time)

# 计算所有点的 x 和 y 坐标
x_values = r_values * np.cos(theta_values_of_each_time)
y_values = r_values * np.sin(theta_values_of_each_time)

# 创建多层列标签，分别表示 x 和 y 坐标
column_labels_x = [f'Point_{i}_x' for i in range(224)]
column_labels_y = [f'Point_{i}_y' for i in range(224)]
column_labels = column_labels_x + column_labels_y

# 合并 x 和 y 数据
combined_values = np.hstack((x_values, y_values))

# 创建 timestamp 列，从 0 到 300，并转换为二维列向量
timestamps = np.arange(301).reshape(-1, 1)

# 将 timestamp 列添加到坐标数据最左侧
combined_values_with_timestamp = np.hstack((timestamps, combined_values))

# 更新列标签，添加 'timestamp' 列
column_labels = ['timestamp'] + column_labels

# 创建 DataFrame
df_coordinates = pd.DataFrame(combined_values_with_timestamp, columns=column_labels)

# 保存到 CSV 文件
df_coordinates.to_csv('./Q1/dragon_coordinates_Q1.csv', index=False)

print("x 和 y 坐标以及时间戳已成功保存到 './Q1/dragon_coordinates_Q1.csv'")


x 和 y 坐标以及时间戳已成功保存到 './Q1/dragon_coordinates_Q1.csv'


### 速度信息

In [238]:
# 创建列标签
column_labels_velocity = [f'Point_{i}_v' for i in range(224)] 

# 创建 timestamp 列
timestamps = np.arange(301)  # 从 0 到 300

# 将 timestamp 列添加到速度数据中
velocity_with_timestamp = np.column_stack((timestamps, velocity_body_values_of_each_time))


# 创建 DataFrame
df_velocity = pd.DataFrame(velocity_with_timestamp, columns=['timestamp'] + column_labels_velocity)

# 保存到 CSV 文件
df_velocity.to_csv('./Q1/dragon_velocity_Q1.csv', index=False)

print("速度信息已成功保存到 './Q1/dragon_velocity_Q1.csv'")


速度信息已成功保存到 './Q1/dragon_velocity_Q1.csv'


# 问题二

### 基本函数

#### 计算垂直于板凳的方向向量的单位向量 $\vec h_n$

In [239]:
def compute_perpendicular_unit_vector(l_n):
    """
    计算垂直于已知向量 l_n 的单位向量 h_n
    输入:
        l_n - 已知的向量（二维或三维）
    输出:
        h_n - 垂直于 l_n 的单位向量
    """
    # 将输入转换为 numpy 数组
    l_n = np.array(l_n)

    # 交换分量并改变符号，得到一个垂直的向量
    h_n = np.array([-l_n[1], l_n[0]])

    # 归一化
    h_n = h_n / np.linalg.norm(h_n)
    
    return h_n

#### 计算初始点的四个顶点坐标

In [241]:
def compute_head_vertices(θ,θ1):
    """
    计算龙头的四个顶点坐标
    输入:
        θ - 龙头的前把手的位置
        θ1 - 第一块龙身的前把手的位置
    输出:
        vertices_0 - 初始四个顶点的坐标列表
        l0 - 龙头的方向向量
        h0 - 垂直于 l0 的单位向量
    """
    # 计算龙头位置的极径
    r = compute_radius(θ)

    # 计算龙头的坐标
    x0 = r * np.cos(θ)
    y0 = r * np.sin(θ)

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    l_0_x, l_0_y= compute_bench_direction_vector(θ, θ1)
    l0 = np.array(compute_unit_vector(l_0_x, l_0_y))

    # 计算垂直于 l0 的单位向量 h0
    h0 = compute_perpendicular_unit_vector(l0)
    
    vertices_0 = [
        np.array([x0, y0]) + 0.15 * h0 + 0.275 * l0,  # 顶点 a1_0
        np.array([x0, y0]) + 0.15 * h0 - 3.135 * l0,  # 顶点 a2_0
        np.array([x0, y0]) - 0.15 * h0 + 0.275 * l0,  # 顶点 a3_0
        np.array([x0, y0]) - 0.15 * h0 - 3.135 * l0,  # 顶点 a4_0
    ]
    return vertices_0, l0, h0

#### 计算当前点的四个顶点坐标

In [242]:
def compute_body_vertices(θ,θ1):
    """
    计算当前龙身的四个顶点坐标
    输入:
        θ - 当前的龙身的前把手的位置
        θ1 - 后一个龙身的前把手的位置
    输出:
        vertices_n - 当前时间点的四个顶点坐标列表
        l_n - 当前向量
        h_n - 当前垂直单位向量
    """

    # 计算当前龙身位置的极径
    r = compute_radius(θ)

    # 计算当前龙身的坐标
    x_n = r * np.cos(θ)
    y_n = r * np.sin(θ)

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    
    l_n_x, l_n_y= compute_bench_direction_vector(θ, θ1)
    l_n = np.array(compute_unit_vector(l_n_x, l_n_y))

    # 计算垂直于 l0 的单位向量 h0
    h_n = compute_perpendicular_unit_vector(l_n)
    
    # 四个顶点的坐标列表
    vertices_n = [
        np.array([x_n, y_n]) + 0.15 * h_n + 0.275 * l_n,  # 顶点 a1_n
        np.array([x_n, y_n]) + 0.15 * h_n - 1.925 * l_n,  # 顶点 a2_n
        np.array([x_n, y_n]) - 0.15 * h_n + 0.275 * l_n,  # 顶点 a3_n
        np.array([x_n, y_n]) - 0.15 * h_n - 1.925 * l_n,  # 顶点 a4_n
    ]
    return vertices_n, l_n, h_n

### 实现分离轴定理 SAT

In [243]:
def projection_interval(vertices, axis):
    """
    计算给定轴上的投影区间。

    参数:
        vertices (list of np.array): 顶点坐标列表。每个顶点是一个 numpy 数组。
        axis (np.array): 投影轴，是一个 numpy 数组。

    返回:
        tuple: 投影区间 (最小值, 最大值)。
    """
    # 计算每个顶点在投影轴上的投影
    projections = [np.dot(vertex, axis) for vertex in vertices]
    
    # 计算投影的最小值和最大值
    min_projection = min(projections)
    max_projection = max(projections)
    
    return min_projection, max_projection

def rectangles_intersect(vertices1, vertices2):
    """
    使用分离轴定理检查两个矩形是否相交。
    参数:
        vertices1 (list of np.array): 第一个矩形的顶点坐标列表。
        vertices2 (list of np.array): 第二个矩形的顶点坐标列表。
    返回:
        bool: 如果两个矩形相交，则返回 True,否则返回 False。
    """
    # print(f"测试 矩阵1: {vertices1} 和 \n 矩阵2: {vertices2}")
    # 获取矩形的两条边（注意：矩形的边是相邻顶点之差）
    edges1 = [vertices1[i] - vertices1[(i + 1) % len(vertices1)] for i in range(2)]  # 只需要前两条边
    edges2 = [vertices2[i] - vertices2[(i + 1) % len(vertices2)] for i in range(2)]

    # 计算所有边的法线向量作为分离轴
    axes = [compute_perpendicular_unit_vector(edge) for edge in edges1 + edges2]

    # 对每个分离轴进行投影并检查是否有重叠
    for axis in axes:
        min1, max1 = projection_interval(vertices1, axis)
        min2, max2 = projection_interval(vertices2, axis)
        if max1 < min2 or max2 < min1:
            return False  # 没有相交

    return True  # 相交

## 分离轴定理 计算是否发生碰撞
只考虑前13个板子（包括龙头）

### 待使用的变量

### 计算前13个点前1000秒的位置坐标

In [245]:
# 计算龙头的所有坐标

# 初始化时间数组
time_array = np.arange(1000)  # 从 0 到 t 的整数数组

# 计算每个时间点的角度值（弧度制）
theta_head_value_2 = np.array([solve_θ_of_dragonhead(t) for t in time_array])

# 计算每个时间点的龙头坐标
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建二维数组来存储每个时间点的所有龙身的 θ 值
theta_body_values_of_each_time_2 = np.empty((1000, 13))  # 301 行，222 列；301个时间点，221个龙身，1个龙尾

# 计算每个时间点的所有龙身的 θ 值
for t in range(1000):
    temp =  theta_head_value_2[t] # 当前时间内点的龙头对应的θ值
    for i in range(13):
        if i == 0:
            theta_body_values_of_each_time_2[t, i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_2[t, i] # 更新当前时间点的龙身对应的θ值
        else:
            theta_body_values_of_each_time_2[t, i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_2[t, i] # 更新当前时间点的龙身对应的θ值


# 将所有时刻的所有点的位置信息存储到同一变量中，301行，13列，第一列为龙头，其余为龙身
theta_values_of_each_time_2 = np.hstack((theta_head_value_2.reshape(-1, 1), theta_body_values_of_each_time_2))

# 创建DataFrame并添加列名
columns = ['Head'] + [f'Body_{i+1}' for i in range(13)]
df = pd.DataFrame(theta_values_of_each_time_2, columns=columns)

# 保存为CSV文件
df.to_csv('./Q2/theta_values_of_each_time_Q2.csv', index=False)

print("CSV文件已成功保存！")

d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


CSV文件已成功保存！


### 情况一：龙头撞龙身

In [246]:
t=-1 # 从 t=300s 开始，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值

while True:
    t = t + 1 # 每一轮结束，t=t+1

    # 计算得到当前时刻的各块的的位置所对应的 θ 值
    
    # 计算得到龙头的相关参数
    θ_0 = theta_values_of_each_time_2[t,0] # 龙头的位置所对应的 theta 值
    θ_1 = theta_values_of_each_time_2[t,1] # 龙头下一个位置（即第一块龙身）所对应的 theta 值

    # 计算得到的四个顶点坐标 和 龙头的方向向量 和 与龙头的方向向量垂直的单位向量 h_0
    vertices_0, l_0, h_0 = compute_head_vertices(θ_0, θ_1)

    for i in range(2, 13): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
        θ_n = theta_values_of_each_time_2[t,i] # 第 i 块龙身所对应的 theta 值
        θ_n1 = theta_values_of_each_time_2[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_n, l_n, h_n = compute_body_vertices(θ_n, θ_n1)

        # 判断是否发生碰撞
        if rectangles_intersect(vertices_0, vertices_n):
            print(f"碰撞发生，t = {t}, 龙头与 块编号为 {i} 发生碰撞" )
            
            break  # 发生碰撞时跳出 for 循环并终止 while 循环
    else:
        # 如果没有碰撞，继续下一个时间点
        continue
    
    # 跳出 while 循环
    break


碰撞发生，t = 413, 龙头与 块编号为 8 发生碰撞


### 情况二：龙身撞龙身

In [247]:
t=-1 # 从 t=300s 开始，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值
flag = 0 # 初始化标志位
while True:
    t = t + 1 # 每一轮结束，t=t+1

    for i in range(1, 13): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
        θ_i = theta_values_of_each_time_2[t,i] # 第 i 块龙身所对应的 theta 值
        θ_i1 = theta_values_of_each_time_2[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_i, l_i, h_i = compute_body_vertices(θ_n, θ_n1)

        for j in range(i+2, 13): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
            # 如果j >=13，则跳出循环,到达边界
            if j >=13:
                break
            θ_j = theta_values_of_each_time_2[t,j] # 第 i 块龙身所对应的 theta 值
            θ_j1 = theta_values_of_each_time_2[t,j+1] # 第 i+1 块龙身所对应的 theta 值

            # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
            vertices_j, l_j, h_j = compute_body_vertices(θ_j, θ_j1)

            # 判断是否发生碰撞
            if rectangles_intersect(vertices_i, vertices_j):
                print(f"碰撞发生，t = {t}, 块编号: {i} 和 {j}")
                flag = 1    # 标志位设为1，表示发生碰撞
        
        if flag == 1: # 如果发生碰撞，跳出 for 循环
            break

    if flag == 1: # 如果发生碰撞，跳出 while 循环
        break



碰撞发生，t = 402, 块编号: 1 和 3


### 计算并导出碰撞前一刻舞龙队的位置和速度
计算得出，当舞龙队发生碰撞时间为`413s`，因为龙身碰龙身的情况有些不合理
故，为使舞龙队不发生碰撞，应在`412s`时停止盘入，下面导出此时舞龙队的位置和速度。

#### 位置信息

计算出 `t=412s` 时，舞龙队的位置信息，用 $\theta$ 表示

In [248]:
# 412s时的龙头前把手位置
θ0_412 = theta_values_of_each_time_2[412,0] # 龙头所对应的 theta 值

# 板凳龙的基本参数
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建列表存储该时间点的所有龙身的 θ 值
theta_body_values_of_412 = np.empty(223)  # 222 个龙身，1个龙尾后把手

# 计算得到此时所有龙身的 θ 值
temp = θ0_412
for i in range(223):
    
    if i == 0:
        theta_body_values_of_412[i] = compute_theta_next(temp, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
        temp = theta_body_values_of_412[i] # 更新当前时间点的龙身对应的θ值
    else:
        theta_body_values_of_412[i] = compute_theta_next(temp, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
        temp = theta_body_values_of_412[i] # 更新当前时间点的龙身对应的θ值

# 合并在 t=412s 时，龙头和龙尾的位置信息
theta_values_of_412 =  np.hstack((θ0_412, theta_body_values_of_412))


将极坐标转换为直角坐标

In [249]:
# 计算每个点的 r 值
r_values = np.vectorize(compute_radius)(theta_values_of_412)

# 计算所有点的 x 和 y 坐标
x_values = r_values * np.cos(theta_values_of_412)
y_values = r_values * np.sin(theta_values_of_412)


# 初始化坐标类型标签和对应的数据列表
labels = []
values = []

# 填充标签和数据，交替放入 x 和 y 值
for i in range(0, 224):  # i 从 1 到 224
    labels.append(f"Point_{i}_x")  # 添加 x 标签
    labels.append(f"Point_{i}_y")  # 添加 y 标签
    values.append(x_values[i])  # 添加对应的 x 值
    values.append(y_values[i])  # 添加对应的 y 值

labels = np.array(labels).flatten()

# 确保 labels 和 values 都是一维列表，且长度匹配
# labels = np.array(labels)  # 转换为一维数组
# values = np.array(values)  # 转换为一维数组

# 确保 labels 和 values 长度相同
print(f"Length of labels: {len(labels)}, Length of values: {len(values)}")

# 创建 DataFrame
df_coordinates = pd.DataFrame({
    "Type": labels,          # 第一列为坐标类型
    "Value": values        # 第二列为对应的 x 或 y 数据
})

# 保存到 CSV 文件
df_coordinates.to_csv('./Q2/dragon_coordinates_Q2.csv', index=False)

print(df_coordinates.head())  # 打印前几行查看结果



Length of labels: 448, Length of values: 448
        Type     Value
0  Point_0_x  2.118587
1  Point_0_y -1.660548
2  Point_1_x  2.530689
3  Point_1_y  1.169606
4  Point_2_x  1.467518


#### 速度

In [251]:
#  speed_of_head 表示每个时刻的龙头的速度大小
speed_of_head= 1

# 建立一个空的数组来存储312s每个点的速度
velocity_values_of_412 = np.empty(224)  # ，224 列；1个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

temp = speed_of_head
velocity_values_of_412[0] = temp
for i in range(0,223):
    velocity_values_of_412[i+1] = compute_velocity_next(theta_values_of_412[i], theta_values_of_412[i+1], temp)
    temp = velocity_values_of_412[i+1]

# 创建点的编号（从 0 开始到 223），共 224 个点
points = np.arange(224)

# 创建 DataFrame，将点编号和速度数据存储为两列
df_velocity = pd.DataFrame({
    "Point": points,  # 第一列为点的编号，范围从 0 到 223
    "Velocity": velocity_values_of_412  # 第二列为对应的速度值
})

# 保存 DataFrame 到 CSV 文件
df_velocity.to_csv('./Q2/dragon_velocity_values_Q2.csv', index=False)

print("CSV 文件已成功保存！")

CSV 文件已成功保存！


# 问题三
在该问题中，螺距`p`为变量

## 基本函数

### 计算极径 r

In [65]:
def compute_radius_3(p, θ):
    """
    根据螺距和角度计算极径。

    参数:
    p (float or np.ndarray): 螺距，单位可以是任意长度单位（如米、毫米等）。
    θ (float or np.ndarray): 角度，以弧度为单位。

    返回:
    float or np.ndarray: 计算得到的极径。如果输入是标量，则返回标量；如果输入是数组，则返回数组。
    """
    # 计算极径
    # 公式: r = 4.5 + p * θ / (2 * π)
    r = 4.5 + p * θ / (2 * np.pi)
    
    # print(f"计算极径 测试 {r}")

    return r

### 计算各点的 $\theta$ 值 

In [66]:
def g_function_3(θ2, θ1, b, p):
    """
    计算方程 g(θ2) - b^2 的值，用于求解 θ2
    输入: theta2 - 待求解的角度 θ2（弧度制）
           theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: g(θ2) - a^2 的值
    """
    r1 = compute_radius_3(p, θ1)
    r2 = compute_radius_3(p, θ2)
    # g(θ2) 的表达式
    g_θ2 = r2**2 - 2 * r1 * r2 * np.cos(θ2 - θ1) + r1**2
    return g_θ2 - b**2

def compute_theta_next_3(θ1, p, b):
    """
    使用数值方法计算 θ2，使得 g(θ2) = b^2 且 θ2 - θ1 ∈ (0, 2π)
    输入: theta1 - 已知角度 θ1（弧度制）
           b - 目标值 b
    输出: 满足条件的 θ2 的值
    """
    # 定义多个初始猜测值，用于 fsolve 寻找不同的解
    initial_guesses = [ θ1+3/(9+6*p), θ1+0.4, θ1+0.5,θ1+2/3]  
    solutions = []

    # 对每个初始猜测值进行求解
    for guess in initial_guesses:
        θ2_solution = fsolve(g_function_3, guess, args=(θ1, p, b))
        θ2 = θ2_solution[0]

        # 检查解是否满足  θ2 - θ1 ∈ (-2π, 0)
        if  0 < (θ2 - θ1) < 2*np.pi:
            solutions.append(θ2)

    # 返回满足条件的最小解
    if solutions:
        unique_solutions = np.unique(solutions)  # 去重
        return min(solutions)
    else:
        return None  # 如果没有满足条件的解，返回 None

### 判断是否发生撞击
参考问题二，使用分离轴定理 SAT 进行判断

In [67]:
def compute_bench_direction_vector_3(θ_n, θ_n1, p):
    """
    计算板凳的方向向量
    输入: 
        r_n, theta_n - 当前点的极径和角度（弧度制）
        r_n1, theta_n1 - 下一个点的极径和角度（弧度制）
    输出: 
        向量 (lx, ly) 
    """
    # r_n 和 r_n1
    r_n = compute_radius_3(p, θ_n)
    r_n1 = compute_radius_3(p, θ_n1)

    # 计算方向向量的 x 和 y 分量
    lx = r_n * np.cos(θ_n) - r_n1 * np.cos(θ_n1)  
    ly = r_n * np.sin(θ_n) - r_n1 * np.sin(θ_n1) 
    
    return lx, ly


In [68]:
def compute_head_vertices_3(θ, θ1, p):
    """
    计算龙头的四个顶点坐标
    输入:
        θ - 龙头的前把手的位置
        θ1 - 第一块龙身的前把手的位置
    输出:
        vertices_0 - 初始四个顶点的坐标列表
        l0 - 龙头的方向向量
        h0 - 垂直于 l0 的单位向量
    """
    # 计算龙头位置的极径
    r = compute_radius_3(p, θ)

    # 计算龙头的坐标
    x0 = r * np.cos(θ)
    y0 = r * np.sin(θ)

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    l_0_x, l_0_y= compute_bench_direction_vector_3(θ, θ1,p)
    l0 = np.array(compute_unit_vector(l_0_x, l_0_y))

    # 计算垂直于 l0 的单位向量 h0
    h0 = compute_perpendicular_unit_vector(l0)
    
    vertices_0 = [
        np.array([x0, y0]) + 0.15 * h0 + 0.275 * l0,  # 顶点 a1_0
        np.array([x0, y0]) + 0.15 * h0 - 3.135 * l0,  # 顶点 a2_0
        np.array([x0, y0]) - 0.15 * h0 + 0.275 * l0,  # 顶点 a3_0
        np.array([x0, y0]) - 0.15 * h0 - 3.135 * l0,  # 顶点 a4_0
    ]
    return vertices_0, l0, h0


## 初始参数设定

In [69]:
p = 1 # 螺距，单位m
θ = 6 * np.pi  # 龙头的初始极角
r = 4.5 + 3*p    # 极径r
# 时间0点为进入掉头空间的时间，t_0 为进入螺线空间的时间 

### 计算节点位置 $\theta$

计算龙头的极角 $\theta$ 的函数

In [70]:
def f(x):
    return 0.5 * x * np.sqrt(x**2 + 1) + 0.5 * np.log(abs(x + np.sqrt(x**2 + 1)))

# 定义 g(x) - t = 0
def equation(x, p, t):
    return (p / (2 * np.pi)) * (f(9 * np.pi / p + 6 * np.pi ) - f(x + 9 * np.pi / p + 6 * np.pi)) - t

def θ_of_dragonhead(t, p, θ):
    """
    解方程 g(x) = t 的 x 值
    输入: t
    输出: 对应的 x 值
    """
    # 初始猜测值为0
    initial_guess = np.radians(0)

    try:
        # 使用 fsolve 求解 g(x) - t = 0 的方程
        x_value = fsolve(equation, x0=initial_guess, args=(p,t))
        return x_value[0] + θ
    except Exception as e:
        print(f"Failed to solve: {e}")
        return None

def compute_body_vertices_3(θ, θ1, p):
    """
    计算当前龙身的四个顶点坐标
    输入:
        θ - 当前的龙身的前把手的位置
        θ1 - 后一个龙身的前把手的位置
    输出:
        vertices_n - 当前时间点的四个顶点坐标列表
        l_n - 当前向量
        h_n - 当前垂直单位向量
    """

    # print(f"测试 vertices_3 θ = {θ}, θ1 = {θ1}")

    # 计算当前龙身位置的极径
    r = compute_radius_3(p, θ)

    # 计算当前龙身的坐标
    x_n = r * np.cos(θ)
    y_n = r * np.sin(θ)

    # print(f"x_n = {x_n}, y_n = {y_n}")

    # 计算垂直于 龙头的方向向量, θ 为龙头的前把手的位置，θ1 为第一块龙身的前把手的位置
    
    l_n_x, l_n_y= compute_bench_direction_vector_3(θ, θ1,p)
    l_n = np.array(compute_unit_vector(l_n_x, l_n_y))

    # 计算垂直于 l0 的单位向量 h0
    h_n = compute_perpendicular_unit_vector(l_n)
    
    # 四个顶点的坐标列表
    vertices_n = [
        np.array([x_n, y_n]) + 0.15 * h_n + 0.275 * l_n,  # 顶点 a1_n
        np.array([x_n, y_n]) + 0.15 * h_n - 1.925 * l_n,  # 顶点 a2_n
        np.array([x_n, y_n]) - 0.15 * h_n + 0.275 * l_n,  # 顶点 a3_n
        np.array([x_n, y_n]) - 0.15 * h_n - 1.925 * l_n,  # 顶点 a4_n
    ]

    # print(f"测试 计算坐标的结果: vertices_n = {vertices_n}")
    # print(f"测试 计算坐标的结果: l_n = {l_n}")
    return vertices_n, l_n, h_n, r 
    

### 判断是否发生撞击

#### 计算前100个时刻 一定数量的点的位置

判断是否碰撞需要考虑几个点

In [71]:
# 设置时间上限
time_limit = 300

# 计算需要判断 n 个点，是否发生碰撞
n = np.ceil( ( 4.45 + 3 * p ) * 2 * np.pi / 1.65 )
n = int(n) # 整数化n
print(f"需要计算n={n}个点") 

需要计算n=29个点


计算前 100s 的 n 个点的运动轨迹

In [72]:
# 初始化时间数组
time_array = np.arange(time_limit)  # 从 0 到 timelimit 的整数数组

# 计算 n 个点的角度值（弧度制）
theta_head_value_n = np.array([θ_of_dragonhead(t, p, θ) for t in time_array])

# 计算每个时间点的龙头坐标
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

# 创建二维数组来存储每个时间点的所有龙身的 θ 值
theta_body_values_of_each_time_n = np.empty((time_limit , n))  

# 计算每个时间点的所有龙身的 θ 值
for t in range(time_limit):
    temp =  theta_head_value_n[t] # 当前时间内点的龙头对应的θ值
    for i in range(n):
        if i == 0:
            theta_body_values_of_each_time_n[t, i] = compute_theta_next_3(temp, p, length_of_dragon_head) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_n[t, i] # 更新当前时间点的龙身对应的θ值
        else:
            theta_body_values_of_each_time_n[t, i] = compute_theta_next_3(temp, p, distance_of_dragon_body) # 当前时间点的龙身每个点的θ值
            temp = theta_body_values_of_each_time_n[t, i] # 更新当前时间点的龙身对应的θ值
# 定义全局变量
global theta_values_of_each_time_n

# 将所有时刻的所有点的位置信息存储到同一变量中，time_limit 行，n 列，第一列为龙头，其余为龙身
theta_values_of_each_time_n = np.hstack((theta_head_value_n.reshape(-1, 1), theta_body_values_of_each_time_n))

# 创建DataFrame并添加列名
columns = ['Head'] + [f'Body_{i+1}' for i in range(n)]
df = pd.DataFrame(theta_values_of_each_time_n, columns=columns)

# 保存为CSV文件
df.to_csv('./Q3/theta_values_of_each_time_Q3.csv', index=False)

print("CSV文件已成功保存！")

d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
d:\conda_envs\ISLP\lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


CSV文件已成功保存！


#### 情景一：龙头撞击龙身

In [73]:
t=-1 # 从 t=0s 开始，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值
flag = 0 # 初始化标志位
while True:
    t = t + 1 # 每一轮结束，t=t+1

    # 避免数组越界
    if t >= time_limit:
        break

    # 计算得到当前时刻的各块的的位置所对应的 θ 值
    
    # 计算得到龙头的相关参数
    θ_0 = theta_values_of_each_time_n[t,0] # 龙头的位置所对应的 theta 值
    θ_1 = theta_values_of_each_time_n[t,1] # 龙头下一个位置（即第一块龙身）所对应的 theta 值

    # 计算得到的四个顶点坐标 和 龙头的方向向量 和 与龙头的方向向量垂直的单位向量 h_0
    vertices_0, l_0, h_0 = compute_head_vertices_3(θ_0, θ_1, p)

    for i in range(2, n): # 遍历前n个龙身(加上龙头，共n个)，排除相邻的块
        θ_n = theta_values_of_each_time_n[t,i] # 第 i 块龙身所对应的 theta 值
        θ_n1 = theta_values_of_each_time_n[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_n, l_n, h_n, r_n = compute_body_vertices_3(θ_n, θ_n1, p)

        # 判断是否发生碰撞
        if rectangles_intersect(vertices_0, vertices_n):
            print(f"碰撞发生，t = {t}, 龙头与 块编号为 {i} 发生碰撞" )
            flag = 1    # 标志位设为1，表示发生碰撞
            
        if flag == 1: # 如果发生碰撞，跳出 for 循环
            break
    
    if flag == 1: # 如果发生碰撞，跳出 while 循环
        break


碰撞发生，t = 0, 龙头与 块编号为 2 发生碰撞


#### 情景二：龙身撞击龙身

In [74]:
t=-1 # 从 t=0s 开始，每一轮结束，t=t+1，直到发生碰撞，弹出循环，返回发生碰撞时的t的值
flag = 0 # 初始化标志位
while True:
    t = t + 1 # 每一轮结束，t=t+1

    # 避免数组越界
    if t >= time_limit:
        break

    for i in range(1,n): # 遍历前0-1个龙身(加上龙头，共n个)，排除相邻的块
        θ_i = theta_values_of_each_time_n[t,i] # 第 i 块龙身所对应的 theta 值
        θ_i1 = theta_values_of_each_time_n[t,i+1] # 第 i+1 块龙身所对应的 theta 值

        # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
        vertices_i, l_i, h_i, r_i = compute_body_vertices_3(θ_n, θ_n1, p)

        # # 打印第一块的坐标
        # if i == 1 :
        #     print(f"第{i}块的坐标",vertices_i)
        #     print(f"第{i}块的r为：", r_i)
        #     print(f"在{t}时间，第{i}块的 θ 值为:",θ_i)

        for j in range(i+2, n): # 遍历前12个龙身(加上龙头，共13个)，排除相邻的块
            # 如果j >=n，则跳出循环,到达边界
            if j >=n:
                break
            
            θ_j = theta_values_of_each_time_n[t,j] # 第 i 块龙身所对应的 theta 值
            θ_j1 = theta_values_of_each_time_n[t,j+1] # 第 i+1 块龙身所对应的 theta 值

            # 计算得到的四个顶点坐标 和 龙身的方向向量 和 与龙身方向向量垂直的单位向量 h_n
            vertices_j, l_j, h_j, r_j = compute_body_vertices_3(θ_j, θ_j1,p)

            

            # 判断是否发生碰撞
            if rectangles_intersect(vertices_i, vertices_j):
                print(f"碰撞发生，t = {t}, 块编号: {i} 和 {j}")
                print(f"第{j}块的坐标为{vertices_j}")
                print(f"第{j}块的r为：", r_j)
                print(f"在{t}时间，第{j}块的 θ 值为:",θ_j)
                flag = 1    # 标志位设为1，表示发生碰撞

        
        if flag == 1: # 如果发生碰撞，跳出 for 循环
            break

    if flag == 1: # 如果发生碰撞，跳出 while 循环
        break



碰撞发生，t = 0, 块编号: 1 和 3
第3块的坐标为[array([7.46572762, 1.92115987]), array([6.77725001, 4.01065708]), array([7.18079618, 1.82727656]), array([6.49231857, 3.91677377])]
第3块的r为： 7.545664433665744
在0时间，第3块的 θ 值为: 19.136474020208038
碰撞发生，t = 0, 块编号: 1 和 4
第4块的坐标为[array([7.23909213, 2.69822956]), array([6.33594262, 4.70430059]), array([6.96553699, 2.57507281]), array([6.06238748, 4.58114384])]
第4块的r为： 7.562350228100081
在0时间，第4块的 θ 值为: 19.24131395863649
碰撞发生，t = 0, 块编号: 1 和 5
第5块的坐标为[array([6.93263063, 3.44678115]), array([5.82527824, 5.34777318]), array([6.67340445, 3.29577855]), array([5.56605205, 5.19677058])]
第5块的r为： 7.5789880640689535
在0时间，第5块的 θ 值为: 19.345852565139367


### 计算碰撞时间的函数化

In [75]:
def find_head_collision_time( n, p, time_limit):
    """
    查找龙头与龙身发生碰撞的时间 t。
    
    参数：
        theta_values_of_each_time_n: 包含每个时间点的 θ 值的二维数组。
        n: 龙身的总节数。
        p: 用于计算顶点和方向向量的参数。
        time_limit: 时间限制，防止无限循环。
    
    返回：
        t_collision: 碰撞发生的时间。如果没有发生碰撞，返回 None。
    """
    t = -1  # 初始化时间
    flag = 0  # 初始化标志位

    # 使用全局变量 theta_values_of_each_time_n
    global theta_values_of_each_time_n

    #print(f"head 碰撞 检测是否传入",theta_values_of_each_time_n)

    while True:
        t = t + 1  # 每一轮结束，t = t + 1

        # 避免数组越界
        if t >= time_limit:
            print("未在指定时间内检测到碰撞")
            return None

        # 计算当前时刻龙头的位置及下一个位置的 θ 值
        θ_0 = theta_values_of_each_time_n[t, 0]  # 龙头的位置所对应的 θ 值
        θ_1 = theta_values_of_each_time_n[t, 1]  # 龙头下一个位置（即第一块龙身）所对应的 θ 值

        # 计算得到的四个顶点坐标和龙头的方向向量，以及与龙头方向向量垂直的单位向量 h_0
        vertices_0, l_0, h_0 = compute_head_vertices_3(θ_0, θ_1, p)

        for i in range(2, n):  # 遍历从第 2 块开始的龙身，排除相邻的块
            θ_n = theta_values_of_each_time_n[t, i]  # 第 i 块龙身所对应的 θ 值
            θ_n1 = theta_values_of_each_time_n[t, i + 1]  # 第 i+1 块龙身所对应的 θ 值
            

            # 计算得到的四个顶点坐标和龙身的方向向量，以及与龙身方向向量垂直的单位向量 h_n
            vertices_n, l_n, h_n, r_n = compute_body_vertices_3(θ_n, θ_n1, p)

            # 判断是否发生碰撞
            if rectangles_intersect(vertices_0, vertices_n):
                print(f"龙头与龙身的碰撞发生，t = {t}, 龙头与块编号为 {i} 发生碰撞")
                
                print(f"θ_n = {θ_n}")
                flag = 1  # 标志位设为 1，表示发生碰撞
                
        
        if flag == 1:  # 发生碰撞后跳出 while 循环
            break

    return t  # 返回发生碰撞的时间


In [76]:
def find_body_collision_time( n, p, time_limit):
    """
    查找碰撞发生的时间 t。
    
    参数：
        theta_values_of_each_time_n: 包含每个时间点的 θ 值的二维数组。
        n: 龙身的总节数。
        p: 用于计算顶点和方向向量的参数。
        time_limit: 时间限制，防止无限循环。
    
    返回：
        碰撞发生的时间 t_collision。如果没有发生碰撞，返回 None。
    """
    t = -1  # 从 t=0s 开始，每一轮结束，t=t+1
    flag = 0  # 初始化标志位

    # 使用全局变量 theta_values_of_each_time_n
    global theta_values_of_each_time_n
    
    # print(f"vody 碰撞 检测是否传入",theta_values_of_each_time_n)
    
    while True:
        t = t + 1  # 每一轮结束，t=t+1

        # 避免数组越界
        if t >= time_limit:
            print("未在指定时间内检测到碰撞")
            return None

        for i in range(1, n):  # 遍历前 0-1 个龙身（加上龙头，共 n 个），排除相邻的块
            θ_i = theta_values_of_each_time_n[t, i]  # 第 i 块龙身所对应的 theta 值
            θ_i1 = theta_values_of_each_time_n[t, i + 1]  # 第 i+1 块龙身所对应的 theta 值

            # print(f"body 测试 θ_i = {θ_i}")
            # 计算得到的四个顶点坐标和龙身的方向向量，以及与龙身方向向量垂直的单位向量 h_n
            vertices_i, l_i, h_i, r_i = compute_body_vertices_3(θ_i, θ_i1, p)

            for j in range(i + 2, n):  # 排除相邻的块
                # 如果 j >= n，则跳出循环，避免越界
                if j >= n:
                    break
                
                θ_j = theta_values_of_each_time_n[t, j]  # 第 j 块龙身所对应的 theta 值
                θ_j1 = theta_values_of_each_time_n[t, j + 1]  # 第 j+1 块龙身所对应的 theta 值

                # print(f"测试 第{i}块龙身所对应的 θ 值：{θ_j}")

                # 计算得到的四个顶点坐标和龙身的方向向量，以及与龙身方向向量垂直的单位向量 h_n
                vertices_j, l_j, h_j, r_j = compute_body_vertices_3(θ_j, θ_j1, p)

                # print(f"测试 第{j}块龙身所对应的 θ 值：{θ_j}")
                # print(f"测试 第{j}块龙身所对应的 坐标 值：{vertices_j}")

                # 判断是否发生碰撞
                if rectangles_intersect(vertices_i, vertices_j):
                    print(f"龙身与龙身的碰撞发生，t = {t}, 块编号: {i} 和 {j}")
                    # # print(f"body 测试 θ_i = {θ_i}")
                    # print(f"body 测试 第{j}块的 theta 值为 {θ_j}")
                    # print(f"第{j}块对应上面一块的坐标为{vertices_j}")
                    # print(f"第{j}块的坐标为{vertices_j}")
                    # print(f"第{j}块的 r 值为：", r_j)
                    # print(f"在 t = {t} 时间，第 {j} 块的 θ 值为:", θ_j)
                    # print(f"l_n = {l_n}")
                    # print(f"h_n = {h_n}")
                    flag = 1  # 标志位设为 1，表示发生碰撞
                    break  # 跳出 j 循环

            if flag == 1:  # 如果发生碰撞，跳出 i 循环
                break

        if flag == 1:  # 如果发生碰撞，跳出 while 循环
            break

    return t  # 返回发生碰撞的时间 t


In [77]:
def collision_time( n, p, time_limit):
    """
    比较得出最早的首次碰撞时间
    """
    t_head_collosion = find_body_collision_time( n, p, time_limit)
    t_body_collosion = find_head_collision_time(n, p, time_limit)

    print(f"情况一：龙头碰龙身,发生在{t_head_collosion}s")
    print(f"情况二：龙头碰龙身,发生在{t_body_collosion}s")
    
    t = min (t_head_collosion, t_body_collosion)

    return t

## 寻找合适的p值

### 设置参数

In [78]:
t_collision = 172     # 发生碰撞的时间

In [79]:
# 计算临界时间
def Cut_off_point(p,x):
    Cut_off_point = ( p / (2 * np.pi) * (f(9 * np.pi / p + 6* np.pi) - f( x + 9 * np.pi / p + 6 * np.pi)) )
    return Cut_off_point

# 用以调整碰撞的临界时间 g(-6π)
cut_off_point = Cut_off_point(p,-6*np.pi)
print(cut_off_point)


113.13797778081573


### 根据碰撞时间调整p值

In [80]:
# 二分法查找临界 p
def find_critical_p(t_collision, cut_off_point, p,  n, time_limit):
    """
    使用二分法查找临界 p，使得 g(p) 接近 t_collision
    输入:
        t_collision - 发生碰撞的时间 t
        Cut_off_point - g(-6π)
        tol - 允许的误差范围，默认为 1/1000
    输出:
        p_final - 使 g(p) 接近 t_collision 的临界 p 值
    """
    # 初始化 p 的区间 [0, 2]
    p_lower, p_upper = 0, 2  # 设置 p 的初始下界和上界
    iteration = 0            # 迭代计数器
    tol = 1/10**3
    
    # 开始二分法循环，直到区间范围小于 tol
    while (p_upper - p_lower) > tol:
        iteration += 1                       # 迭代次数累加
        p_mid = (p_lower + p_upper) / 2      # 计算 p 的中间值

        # 判断 g(-6π) 和 t_collision 的关系以调整 p 的范围
        if cut_off_point > t_collision:
            # 如果 g(-6π) 大于 t_collision，则碰撞时间过早，需要 p 值 变大
            p_lower = p
            p_mid = (p_lower + p_upper) / 2
            p = p_mid
 
        else:
            # 如果 g(-6π) 小于等于 t_collision，则碰撞时间过早，需要减小 p 值
            p_upper = min(p,p_upper)
            p_mid = (p_upper + p_lower) / 2
            p = p_mid

        t_collision = collision_time( n, p, time_limit) # 计算得出修改后的碰撞时间
        cut_off_point = Cut_off_point(p,-6*np.pi) # g(-6π) 跟随发生变化

        # 输出当前迭代的详细信息
        print(f"Iteration {iteration}: p_mid = {p_mid}, p_upper = {p_upper}, p_lower = {p_lower}, t_collison = {t_collision}, p = {p}, g(-6pi)={cut_off_point}")

    # 返回最终找到的临界 p 值
    p_final = (p_lower + p_upper) / 2        # 在最终区间中取平均值作为结果
    return p_final

In [81]:
critical_p = find_critical_p(t_collision, cut_off_point, p,  n, time_limit)
print(f"临界的 p 值为: {critical_p}")

龙身与龙身的碰撞发生，t = 0, 块编号: 1 和 3
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 2 发生碰撞
θ_n = 19.03133012882195
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 3 发生碰撞
θ_n = 19.136474020208038
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 4 发生碰撞
θ_n = 19.24131395863649
情况一：龙头碰龙身,发生在0s
情况二：龙头碰龙身,发生在0s
Iteration 1: p_mid = 0.5, p_upper = 1, p_lower = 0, t_collison = 0, p = 0.5, g(-6pi)=98.97161441366771
龙身与龙身的碰撞发生，t = 0, 块编号: 1 和 3
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 2 发生碰撞
θ_n = 19.03133012882195
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 3 发生碰撞
θ_n = 19.136474020208038
情况一：龙头碰龙身,发生在0s
情况二：龙头碰龙身,发生在0s
Iteration 2: p_mid = 0.75, p_upper = 1, p_lower = 0.5, t_collison = 0, p = 0.75, g(-6pi)=106.05294855921086
龙身与龙身的碰撞发生，t = 0, 块编号: 1 和 3
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 2 发生碰撞
θ_n = 19.03133012882195
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 3 发生碰撞
θ_n = 19.136474020208038
情况一：龙头碰龙身,发生在0s
情况二：龙头碰龙身,发生在0s
Iteration 3: p_mid = 0.875, p_upper = 1, p_lower = 0.75, t_collison = 0, p = 0.875, g(-6pi)=109.59503615325185
龙身与龙身的碰撞发生，t = 0, 块编号: 1 和 3
龙头与龙身的碰撞发生，t = 0, 龙头与块编号为 2 发生碰撞
θ_n = 19.03133012882195
龙头与龙身的碰

# 问题四

## 导入库

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad

## 基础公式

### 新的螺线极坐标方程
方程编号 m1

In [ ]:
def m1(theta):
    """
    输入 θ，返回新的螺线极坐标方程中的 极径 r
    """
    # 确保 theta 是浮点数
    theta = float(theta)
    r = 4.5 + 1.7 * theta / ( 2 * np.pi )
    return r

m1 的导数的值

In [ ]:
# m1 的导数
dr = 1.7 / ( 2 * np.pi )

# 打印 m1 的导数
print("m1 的导数：", dr)

m1 的导数： 0.2705634032562221


计算常量 alpha 的值

In [ ]:
global alpha
alpha = np.arctan( m1(0) / np.sqrt( m1(0)**2 + dr**2 ) ) # 计算角度 α

# 打印角度 α
print("弧度 α：", alpha)

弧度 α： 0.7844960335841645


### 圆 d 的极坐标方程
方程编号 m2

In [ ]:
# 求根公式求解m2，用在n_1中
def m2(theta):
    """
    利用求根公式计算给定的 theta 和 alpha 
    """
    # 确保 theta 是浮点数
    theta = float(theta)
    global alpha

    term1 = 4 * ( np.sin(alpha) ** 2) + ( np.cos(alpha) ** 2 ) 
    term2 = theta - np.arctan( 1 / 2*np.tan(alpha) )

    part1 = np.sqrt( term1 ) * np.cos( term2 )
    part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )
    part3 = 3 / ( 2 * np.sin(alpha) ) 

    return (part1 + part2) * part3

# 求根公式求解m2，用在n_2中
def m3(theta):
    """
    利用求根公式计算给定的 theta 和 alpha 
    """
    # 确保 theta 是浮点数
    theta = float(theta)
    
    global alpha

    term1 = 4 * ( np.sin(alpha) ** 2) + ( np.cos(alpha) ** 2 ) 
    term2 = theta - np.arctan( np.tan(alpha) / 2 )

    part1 = np.sqrt( term1 ) * np.cos( term2 )
    part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )
    part3 = 3 / ( 2 * np.sin(alpha) ) 

    return ( part1 - part2 ) * part3

### 圆 e 的极坐标方程
方程编号 m3

In [ ]:
def m4(theta):
    """
    利用求根公式计算 m3 的值
    """
    global alpha

    # 确保 theta 是浮点数
    theta = float(theta)

    tan_alpha = np.tan(alpha)
    sin_alpha = np.sin(alpha)

    subterm1 = 3 / tan_alpha
    subterm4 = subterm1**2
    subterm2 =  9 + ( subterm4 )
    subterm3 = (3 / sin_alpha)**2
    

    term1 = np.sqrt ( subterm2 )
    term2 = np.cos( theta - np.arctan( 2 * tan_alpha ) )
    
    part1 = term1 * term2
    part2 = np.sqrt(subterm2 * term2 - subterm4 - 2.25 + subterm3)

    return part1 + part2

### 盘出螺线的极坐标方程
方程编号 m4

In [ ]:
def m5(θ):
    """
    计算盘出螺线的极坐标方程 r_4 = 4.5 + (1.7 / (2 * pi)) * theta - (1.7 / 2)。

    参数:
        theta (float or np.array): 极角 θ 的值，可以是单个数值或数组。
        p (float): 调整螺线的参数。

    返回:
        float or np.array: 螺线在给定 θ 下的半径 r_4。
    """
    # 计算螺线的半径
    r = 4.5 + (1.7 / (2 * np.pi)) * θ - (1.7 / 2)
    return r

## 计算余弦函数

In [ ]:
def cosine(i, j, theta_0, theta_1):
    
    theta_0 = float(theta_0)
    theta_1 = float(theta_1)
    values_a = {
        1: m1(theta_0),
        2: m2(theta_0),
        3: m3(theta_0),
        4: m4(theta_0),
        5: m5(theta_0)
    }
    a = values_a.get(i, "i is not in the range 1 to 5")

    values_b = {
        1: m1(theta_1),
        2: m2(theta_1),
        3: m3(theta_1),
        4: m4(theta_1),
        5: m5(theta_1)
    }
    b = values_b.get(j, "j is not in the range 1 to 5")

    a = float(a)
    b = float(b)    

    return a ** 2 + b ** 2 - 2 * a * b * np.cos(theta_0 - theta_1)

### 定值 $\beta$ 的值

In [ ]:
global beta
beta = np.arcsin( 1 / np.sqrt(3 * (np.sin(alpha)**2) + 1) )

print(f"beta 的值: {beta}")

beta 的值: 0.6851615933712729


### 分段函数 $n_1(\theta)$

In [ ]:
def n1_theta(theta):
    """
    根据给定的 theta 和 beta 计算 n1(theta)。

    参数:
        theta (float): 输入角度 theta。
        beta (float): 参数 beta。
    
    返回:
        float: 根据区间选择的 m1(theta) 或 m2(theta) 的计算结果。
    """
    global beta

    # 判断 theta 是否为空或者超出范围
    if theta is None or theta < - beta:
        print(f"警告: theta 值为空或超出范围，设定为 1")
        theta = 1  # 将 theta 赋值为 1

    if theta >= 0 :
        return m1(theta)  # 调用 m1 函数
    elif -np.pi/2-beta <= theta < 0:
        return m2(theta)  # 调用 m2 函数
    else:
        print(theta)
        raise ValueError("theta 超出了 n1(theta) 定义的范围")

分段函数 $n_2(\theta)$

In [ ]:
def n2_theta(theta):
    """
    根据给定的 theta 和 beta 计算 n2(theta)。

    参数:
        theta (float): 输入角度 theta。
        beta (float): 参数 beta。
    
    返回:
        float: 根据区间选择的 m2(theta), m3(theta) 或 m4(theta) 的计算结果。
    """
    global beta

    if -np.pi/2-beta <= theta < 0:
        return m3(theta)  # 调用 m2 函数
    elif 0 <= theta < np.pi:
        return m4(theta)  # 调用 m3 函数
    elif theta >= np.pi:
        return m5(theta)  # 调用 m4 函数
    else:
        print(theta)
        raise ValueError("theta 超出了 n2(theta) 定义的范围")

### 极角为 $\theta$ , 表示距离起点的弧长公式 g($\theta$)

In [ ]:
def g(theta):
    """
    计算函数 g(θ) 的值。

    参数:
    theta (float): 输入变量 θ。
    p (float): 参数 p。

    返回值:
    float: 计算得到的 g(θ) 值。
    """
    p = 1.7 

    # 计算常数项
    term1 = (9 * np.pi) /p
    sqrt_term1 = np.sqrt(term1**2 + 1)
    
    # 计算第一组项
    part1 = (9 * np.pi /2*p) * sqrt_term1
    part2 = 0.5 * np.log(abs(term1 + sqrt_term1))
    
    # 计算第二组项
    term2 = theta + term1
    sqrt_term2 = np.sqrt(term2**2 + 1)
    part3 = 0.5 * term2 * sqrt_term2
    part4 = 0.5 * np.log(abs(term2 + sqrt_term2))
    
    # 合并所有项
    g_value = (p / (2 * np.pi)) * ( part3 + part4-part1-part2)
    return g_value


In [ ]:
import numpy as np
from scipy.integrate import quad
from scipy.optimize import approx_fprime

def integrand(theta):
    """
    被积函数，用于计算弧长
    """
    r = m2(theta)
    dr_dtheta = approx_fprime(np.array([theta]), lambda x: m2(x), epsilon=1e-8)
    
    return np.sqrt(r**2 + dr_dtheta[0]**2)

def equation_arc(theta, t):

    arc_length, error = quad(integrand, 0, theta)

    return arc_length - t


### 在不同的时间段求解 $\theta_0$ 的值的公式
$\theta_0$ 表示龙头的位置信息

In [ ]:
# 定义反解 g(θ) = t + 100 的方程
def inverse_g_equation(theta, t):
    return g(theta) + t

# 定义求解 theta 的函数
def solve_head_theta(t):
    """
    根据给定的时间 t、alpha 和 beta 求解对应的 theta。

    参数:
        t (float): 时间变量。
        alpha (float): 常量 alpha。
        beta (float): 常量 beta。

    返回:
        float: 求解得到的 theta 值。
    """

    global alpha
    global beta

    gamma = 0

    # 判断时间段并计算 theta

    if t <= 0:
        gamma = 1 
        # t < 0 时, 反解出 θ0
        initial_guess_1 = 33+t/3  # 初始猜测值
        theta_solution = fsolve(inverse_g_equation, initial_guess_1, args=(t,))
        return theta_solution[0], gamma

    elif 0 < t < (np.pi / 2) + beta:
        gamma = 2
        initial_guess_2 = 0.5 
        # 0 ≤ t < π/2 + β 时, 计算 θ
        part1 = 3 + (1.5 / np.sin(alpha)) * np.cos(np.pi/2 + alpha + 2 * t * np.sin(alpha))
        part2 = (1.5 / np.sin(alpha)) * np.sin(np.pi/2 + alpha + 2* t * np.sin(alpha)/3) - 1.5 / np.tan(alpha)
        theta =  part1 / part2
        return theta, gamma

    elif ( np.pi / 2 + beta) * 1.5 / np.sin(alpha) <= t < (3 * alpha) / np.sin(alpha):

        gamma = 3
        # π/2 + β ≤ t < 3α/sinα 时, 计算 θ
        part1 = 3 + (1.5 / np.sin(alpha)) * np.cos(np.pi/2 + alpha + 2 * t * np.sin(alpha))
        part2 = (1.5 / np.sin(alpha)) * np.sin(np.pi/2 + alpha + 2* t * np.sin(alpha)/3) - 1.5 / np.tan(alpha)
        theta =  - part2 / part1
        return theta, gamma

    elif (3 * alpha) / np.sin(alpha) <= t < (9 * alpha) / np.sin(alpha):

        gamma = 4
        # 3α/sinα ≤ t < 9α/sinα 时, 计算 θ
        term1 = 3* alpha / np.sin(alpha)
        term2 = 3 / np.sin(alpha)
        term3 = alpha - (t - term1) / term2
        part1 = term1 * np.cos(term3) - 3/np.tan(alpha)
        part2 = 1.5 - term1 * np.sin(term3)

        theta = np.pi - np.arctan(part1 / part2)
        return theta, gamma

    else:
        gamma = 5
        # 9α/sinα ≤ t 时, 反解出 θ
        def equation_for_theta(theta):
            return g(theta) - g(3 * alpha) - (t - (9 * alpha) / np.sin(alpha))

        initial_guess = 3* alpha + t* (9/2)  # 初始猜测值，需根据 g(θ) 的性质调整
        theta_solution = fsolve(equation_for_theta, initial_guess)
        return float(theta_solution[0]), gamma


### 求不同时间点的所有点的 $\theta$ 的值

In [ ]:
length_of_dragon_head = 2.86 # 龙头的长度 286cm=2.86m
distance_of_dragon_body= 1.65 # 龙身上两个孔的距离 165cm=1.65m

In [ ]:
# 判断 当前点与 pi/2+beta*1.5/3alpha 的大小关系
def compare_1(x):
    global alpha
    global beta
    
    if x < ( np.pi / 2 + beta) * 1.5 / np.sin(alpha) :
        return 1 # 返回 1
    if x > ( np.pi / 2 + beta) * 1.5 / np.sin(alpha):
        return 2 # 返回 2 
    
def compare_2(x):
    global alpha
    global beta
    
    if x < -( np.pi / 2 + beta) * 1.5 / np.sin(alpha) :
        return 1 # 返回 1
    if x > -( np.pi / 2 + beta) * 1.5 / np.sin(alpha):
        return 2 # 返回 2 
    
def compare_3(theta):
    if theta is None:
        return 1
    else:
        return 2
    
def fun_1(theta_1, i, j, theta_0, rsh):

    return cosine(i, i,theta_0, theta_1) - rsh




In [ ]:
# 计算当前时间点 t 的所有点的 theta 值
def theta_next_function_1(theta_1, theta_0, rhs):
    
    n1_theta_n = n1_theta(theta_0)
    n1_theta_n1 = n1_theta(theta_1)
    
    return n1_theta_n ** 2 + n1_theta_n1 ** 2 - 2 * n1_theta_n * n1_theta_n1 * np.cos( theta_0 - theta_1 ) - rhs

def theta_next_function_2(theta_1, theta_0, rhs):
    
    n2_theta_n = n2_theta(theta_0)
    n2_theta_n1 = n2_theta(theta_1)
    
    return n2_theta_n ** 2 + n2_theta_n1 ** 2 - 2 * n2_theta_n * n2_theta_n1 * np.cos( theta_0 - theta_1 ) - rhs

def theta_next_function_3(theta_1, theta_0, rhs):
    
    global beta
    # (-beta, 0 )
    
    if theta_1 > - beta and theta_1 < 0:
        n1_theta_n1 = m2(theta_1)
    else:
        n1_theta_n1 = m1(theta_1)

    n2_theta_n = n2_theta(theta_0)
    
    return n2_theta_n ** 2 + n1_theta_n1 ** 2 - 2 * n2_theta_n * n1_theta_n1 * np.cos( theta_0 - theta_1 ) - rhs

def find_min_solution_in_range(theta_0, solutions, lower_bound=0, upper_bound=2 * np.pi):
    """
    过滤解在指定范围内并找到最小的解

    参数:
    - solutions: 解的列表
    - lower_bound: 范围的下界，默认为 0
    - upper_bound: 范围的上界，默认为 2 * np.pi

    返回:
    - theta_min: 最小的符合范围的解，如果没有找到符合条件的解，返回 None
    """
    # 过滤在指定范围内的解
    valid_solutions = [theta for theta in solutions if lower_bound < theta - theta_0 < upper_bound]

    # 找出最小的解
    if valid_solutions:
        theta_min = min(valid_solutions)
        return theta_min
    else:
        print("没有找到符合范围的解")
        return None
    
def find_min_solution_in_range_1(theta_0, solutions, lower_bound=0, upper_bound=2 * np.pi):
    """
    过滤解在指定范围内并找到最小的解

    参数:
    - solutions: 解的列表
    - lower_bound: 范围的下界，默认为 0
    - upper_bound: 范围的上界，默认为 2 * np.pi

    返回:
    - theta_min: 最小的符合范围的解，如果没有找到符合条件的解，返回 None
    """

    global beta

    # 过滤在指定范围内的解
    valid_solutions = [theta for theta in solutions if lower_bound < theta + beta + np.pi/2 < upper_bound]

    # 找出最小的解
    if valid_solutions:
        theta_min = min(valid_solutions)
        return theta_min
    else:
        print("没有找到符合范围的解")
        return None
    
def find_max_solution_in_range(theta_0, solutions, lower_bound=0, upper_bound=2 * np.pi):
    """
    过滤解在指定范围内并找到最大的解

    参数:
    - solutions: 解的列表
    - lower_bound: 范围的下界，默认为 0
    - upper_bound: 范围的上界，默认为 2 * np.pi

    返回:
    - theta_min: 最大的符合范围的解，如果没有找到符合条件的解，返回 None
    """
    # 过滤在指定范围内的解
    valid_solutions = [theta for theta in solutions if lower_bound < theta - theta_0 < upper_bound]

    # 找出最大的解
    if valid_solutions:
        theta_max = max
    else:
        print("没有找到符合范围的解")
        return None

def calculate_theta_next(theta_0, theta_1, i, t, a, b, gamma_of_each_time):

    global alpha
    global beta

    gamma = gamma_of_each_time[t, i]

    # 初始猜测值
    initial_guess_1 = [ theta_0 + 0.1,theta_0+0.3 ] #(0, 2pi) 的最小值
    initial_guess_2 = [ theta_0+1.5 , theta_0 + 1, theta_0+2, theta_0+0.3 ] 
    initial_guess_3 = [ theta_0 , theta_0 - 2 * np.sin(alpha)] # (-2pi, 0) 取最大值
    initial_guess_4 = [ theta_0 , theta_0 - np.sin(alpha)]
    initial_guess_5 = [ theta_0 - 0.5, theta_0 + 0.5 ] 


    guess_n = {
        1: initial_guess_1,
        2: initial_guess_2,
        3: initial_guess_3,
        4: initial_guess_4,
        5: initial_guess_5
    }
    
    guess_num = guess_n.get(gamma, "i is not in the range 1 to 5")

    
    # 根据 i 的值选择方程右边的常数 a² 或 b²
    if i == 0:
        rhs = a**2
    else:
        rhs = b**2

    # 求解 theta_n1
    solutions = []
    for guess in guess_num:
        sol = fsolve(fun_1, guess, args=(gamma,gamma, theta_0, rhs))
        solutions.append(sol[0])  # fsolve 返回的是数组，取第一个元素
    if gamma in [1, 2]:
        theta_next = find_min_solution_in_range(theta_0 , solutions, 0, 2 * np.pi)
    elif gamma in [3, 4, 5]:
        theta_next = find_max_solution_in_range(theta_0, solutions, - 2 * np.pi, 0)

    # 对theta_n1进行判断
    intervals = {
        1: ( 0, 100 ),   
        2: ( - np.pi / 2 + beta, 0 ),  
        3: ( np.pi / 2 + beta, 0 ),  
        4: ( 0, np.pi ),  
        5: ( np.pi, 100 )   

    }
    start, end = intervals[gamma]
    if start <= theta_0 <= end:
        gamma_of_each_time[t, i+1] = gamma
        print(f"t={t},i={i},gamma={gamma},theta_n={theta_0},theta_next={theta_next}")
        return float(str(theta_next))
    else:
        gamma_of_each_time[t, i+1] = gamma +1
        solutions = []
        guess_num = guess_n.get( gamma+1, "i is not in the range 1 to 5")
        for guess in guess_num:
            sol = fsolve(fun_1, guess, args=(gamma, gamma+1, theta_0, rhs))
            solutions.append(sol[0])  # fsolve 返回的是数组，取第一个元素
        if gamma in [1, 2]:
            theta_next = find_min_solution_in_range(theta_0, solutions, 0, 2 * np.pi)
        elif gamma in [3, 4, 5]:
            theta_next = find_max_solution_in_range(theta_0, solutions, - 2 * np.pi, 0)
        print(f"t={t},i={i},gamma={gamma},theta_next={theta_0}")
        return float(str(theta_next))

### 将极坐标转换为直角坐标

In [ ]:
# 定义计算 x_i 和 y_i 的函数
def compute_coordinates(theta, t, i, gamma_of_each_time):
    """
    根据给定的 theta_i 计算 x_i 和 y_i。
    将极坐标转换为直角坐标

    参数:
        theta_i (float): 输入角度 θ_i，以弧度为单位。

    返回:
        tuple: 计算得到的 (x_i, y_i) 坐标。
    """
    gamma = gamma_of_each_time[t, i]
    if gamma == 1:
        n_gamma = n1_theta(theta)
    if gamma == 2:
        n_gamma = n2_theta(theta)
        
    x_i = n_gamma * np.cos(theta)
    y_i = n_gamma * np.sin(theta)
    
    return x_i, y_i


### 求所有点的单位方向向量 $\vec l$

In [ ]:
def unit_vector(v):
    """
    计算给定向量的单位向量。
    
    参数:
        v (array-like): 输入向量
    
    返回:
        ndarray: 单位向量
    """
    v = np.array(v)  # 将输入转换为 NumPy 数组
    norm = np.linalg.norm(v)  # 计算向量的模（或长度）
    
    if norm == 0:
        raise ValueError("零向量没有单位向量")
    
    return v / norm  # 返回单位向量

def unit_direction_vector(theta_0, theta_1, t, i, gamma_of_each_time):
    """
    求单位方向向量
    """
    x_0, y_0 = compute_coordinates(theta_0, t, i, gamma_of_each_time)
    x_1, y_1 = compute_coordinates(theta_1, t, i+1, gamma_of_each_time) 

    ln = np.array([x_0 - x_1, y_0 - y_1])
    l = unit_vector(ln)

    return l

### 求现有点的速度大小

In [ ]:
def velocity_value(theta_0, theta_1, t, i, gamma_of_each_time):
    """
    计算该点的速度大小
    """


### 新版的公式

## 位置信息

### 计算每个时刻的每个点的位置信息

In [ ]:
# 用两个二维向量来存储每个点的信息
gamma_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 gamma 值
theta_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 theta 值

# 计算所有时间点的所有龙头的位置信息
for t in range(-100,101):
    theta_of_each_time[t,0], gamma_of_each_time[t,0] = solve_head_theta(t)
    if t == 1:
        print(f"t=1，时，theta_0={theta_of_each_time[1,0]}")


a=2.86
global alpha
global beta
gamma=0
theta_1=1
b = distance_of_dragon_body
theta_0 = 1
result = m1(theta_0) ** 2 + m2(theta_1) ** 2 - 2*m1(theta_0) * m2(theta_1) * np.cos(theta_0 - theta_1) - b ** 2 
print(result,theta_of_each_time[0, 0])

t=1，时，theta_0=-1.516344945689631
-2.315252900602349 11.649685103315038


In [ ]:
# 用两个二维向量来存储每个点的信息
gamma_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 gamma 值
theta_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 theta 值

# 计算所有时间点的所有龙头的位置信息
for t in range(-100,101):
    theta_of_each_time[t,0], gamma_of_each_time[t,0] = solve_head_theta(t)

global alpha
global beta

gamma = gamma_of_each_time[-100, 0]
theta_0 = theta_of_each_time[-100, 0]
b = distance_of_dragon_body
# 初始猜测值
initial_guess_1 = [ theta_0 + 0.1, theta_0 + 3 + 2 * 1.7, theta_0+1.5 ] #(0, 2pi) 的最小值

# 求解 theta_n1
solutions = []
for guess in initial_guess_1:
    sol = fsolve(fun_1, guess, args=(1,1, theta_0, b**2))
    solutions.append(sol[0])  # fsolve 返回的是数组，取第一个元素
if gamma in [1, 2]:
    theta_next = find_min_solution_in_range(theta_0 , solutions, 0, 2 * np.pi)
    print(theta_next)
elif gamma in [3, 4, 5]:
    theta_next = find_max_solution_in_range(theta_0, solutions, - 2 * np.pi, 0)
    print(theta_next)

22.749859766792756


C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2944921336.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  theta_1 = float(theta_1)
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2584618864.py:23: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt(subterm2 * term2 - subterm4 - 2.25 + subterm3)
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2677905717.py:14: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2677905717.py:33: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )


In [ ]:
# 用两个二维向量来存储每个点的信息
gamma_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 gamma 值
theta_of_each_time = np.empty((201, 224), dtype=float) # 创建一个形状为 [201, 224] 的全 0 的二维数组,用以存储每一时刻的每一个点的 theta 值

# 计算所有时间点的所有龙头的位置信息
# -100，101
for t in range(-1,101):
    theta_of_each_time[t,0], gamma_of_each_time[t,0] = solve_head_theta(t)

# 计算所有时间点的所有点的所有位置信息
for t in range(-1,101):
    for i in range(0,222):
        print(t,i)
        theta_of_each_time[t, i+1] = calculate_theta_next(theta_of_each_time[t,i], theta_of_each_time[t,i+1], i, t, length_of_dragon_head, distance_of_dragon_body, gamma_of_each_time)


-1 0
t=-1,i=0,gamma=1.0,theta_n=11.779988838929748,theta_next=12.151530174291734
-1 1
t=-1,i=1,gamma=1.0,theta_n=12.151530174291734,theta_next=12.36289387161264
-1 2
t=-1,i=2,gamma=1.0,theta_n=12.36289387161264,theta_next=12.57272404874793
-1 3
t=-1,i=3,gamma=1.0,theta_n=12.57272404874793,theta_next=12.781053635537962
-1 4
t=-1,i=4,gamma=1.0,theta_n=12.781053635537962,theta_next=12.987914399315908
-1 5
t=-1,i=5,gamma=1.0,theta_n=12.987914399315908,theta_next=13.193337001584808
-1 6
t=-1,i=6,gamma=1.0,theta_n=13.193337001584808,theta_next=13.397351051189615
-1 7
t=-1,i=7,gamma=1.0,theta_n=13.397351051189615,theta_next=13.599985154245644
-1 8
t=-1,i=8,gamma=1.0,theta_n=13.599985154245644,theta_next=13.801266961062087
-1 9
t=-1,i=9,gamma=1.0,theta_n=13.801266961062087,theta_next=14.001223210278761
-1 10
t=-1,i=10,gamma=1.0,theta_n=14.001223210278761,theta_next=14.1998797704158
-1 11
t=-1,i=11,gamma=1.0,theta_n=14.1998797704158,theta_next=14.39726167901927
-1 12
t=-1,i=12,gamma=1.0,theta_n

C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2944921336.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  theta_1 = float(theta_1)
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2677905717.py:14: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2677905717.py:33: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt( term1 * ( np.cos( term2 )**2 ) + 1 - term1 )
C:\Users\Austin\AppData\Local\Temp\ipykernel_45152\2584618864.py:23: RuntimeWarning: invalid value encountered in sqrt
  part2 = np.sqrt(subterm2 * term2 - subterm4 - 2.25 + subterm3)


t=-1,i=190,gamma=1.0,theta_n=39.35984009115321,theta_next=39.46868646306736
-1 191
t=-1,i=191,gamma=1.0,theta_n=39.46868646306736,theta_next=39.57732191908775
-1 192
t=-1,i=192,gamma=1.0,theta_n=39.57732191908775,theta_next=39.68574768085573
-1 193
t=-1,i=193,gamma=1.0,theta_n=39.68574768085573,theta_next=39.79396495826328
-1 194
t=-1,i=194,gamma=1.0,theta_n=39.79396495826328,theta_next=39.901974949610484
-1 195
t=-1,i=195,gamma=1.0,theta_n=39.901974949610484,theta_next=40.00977884176092
-1 196
t=-1,i=196,gamma=1.0,theta_n=40.00977884176092,theta_next=40.117377810293625
-1 197
t=-1,i=197,gamma=1.0,theta_n=40.117377810293625,theta_next=40.22477301965275
-1 198
t=-1,i=198,gamma=1.0,theta_n=40.22477301965275,theta_next=40.331965623294664
-1 199
t=-1,i=199,gamma=1.0,theta_n=40.331965623294664,theta_next=40.438956763832564
-1 200
t=-1,i=200,gamma=1.0,theta_n=40.438956763832564,theta_next=40.5457475731787
-1 201
t=-1,i=201,gamma=1.0,theta_n=40.5457475731787,theta_next=40.65233917268414
-1 20

ValueError: could not convert string to float: 'None'

### 位置信息导入表格

In [ ]:
# 初始化空列表来存储坐标值
coordinates_list = []

# 计算每个 t 和 i 的 x, y 坐标
for t in range(-100, 101):
    for i in range(0, 224):
        # 计算 x 和 y 值
        x_value, y_value = compute_coordinates(theta_of_each_time[t, i], t, i, gamma_of_each_time)
        # 将 t, i, x_value, y_value 作为一行数据存入列表
        coordinates_list.append([t, i, x_value, y_value])

# 将数据转换为 Pandas DataFrame
df = pd.DataFrame(coordinates_list, columns=["t", "i", "x", "y"])

# 将 DataFrame 保存为 CSV 文件
df.to_csv("./Q4/coordinates_Q4.csv", index=False)

print("坐标数据已保存到 coordinates_Q4.csv 文件")

TypeError: compute_coordinates() takes 1 positional argument but 4 were given

## 速度信息

### 计算所有时间点的所有点的所有位置信息

In [ ]:
def compute_velocity_next(θ_n, θ_n1, u_n):
    """
    计算速度大小 u_{n+1}
    
    参数：
    u_n: 当前点速度的大小
    v_n: 当前点的速度方向向量
    v_n1: 下一点的速度方向向量
    l: 参考方向的单位向量
    
    返回：
    u_{n+1}: 下一点的速度大小
    """

    # 计算速度方向向量
    v_n = compute_unit_vector(*compute_velocity(θ_n))
    v_n1 = compute_unit_vector(*compute_velocity(θ_n1))

    # 计算板凳的方向向量
    l_n_x, l_n_y = compute_bench_direction_vector(θ_n, θ_n1)
    l_n = np.array([l_n_x, l_n_y])
    
    # 计算点乘
    numerator = np.dot(v_n, l_n) # 计算分子
    denominator = np.dot(v_n1, l_n)   # 计算分母
    
    # 确保分母不为零
    if denominator == 0:
        raise ValueError("Denominator is zero, cannot divide.")
    
    # 更新速度大小
    u_n1 = (numerator / denominator) * u_n

    return u_n1


In [ ]:
# 建立一个空的数组来存储每个时间点龙身每个点的速度
velocity_body_values_of_each_time = np.empty((301, 224))  # 301 行，223 列；301个时间点，一个龙头，221个龙身，1个龙尾，最后一列为龙尾后把手

for t in range(301):
    temp = speed_of_head # 计算当前时间点的龙头速度大小
    velocity_body_values_of_each_time[t, 0] = temp # u0 存放龙头的速度大小
    for i in range(0,223):
        velocity_body_values_of_each_time[t, i+1] = compute_velocity_next(theta_values_of_each_time[t,i], theta_values_of_each_time[t,i+1], temp)
        temp = velocity_body_values_of_each_time[t, i+1]

### 速度信息导入表格

In [ ]:
# 创建列标签
column_labels_velocity = [f'Point_{i}_v' for i in range(224)] 

# 创建 timestamp 列
timestamps = np.arange(301)  # 从 0 到 300

# 将 timestamp 列添加到速度数据中
velocity_with_timestamp = np.column_stack((timestamps, velocity_body_values_of_each_time))


# 创建 DataFrame
df_velocity = pd.DataFrame(velocity_with_timestamp, columns=['timestamp'] + column_labels_velocity)

# 保存到 CSV 文件
df_velocity.to_csv('./Q4/dragon_velocity_Q4.csv', index=False)

print("速度信息已成功保存到 './Q4/dragon_velocity_Q4.csv'")

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 301 and the array at index 1 has size 1000

# 问题五

In [ ]:
import random

# 初始解 v_0'
initial_solution = 1.7 # 单位 1.7m/s

# 目标函数，假设 E(v_n') 是一个你提供的函数，这里是占位符
def objective_function(v):
    # E(v_n') 计算把手的最大速度 (此处用一个假设的函数表示)
    return v * (2 - v / 10)  # 这个公式可以替换为你具体的模型

# 生成邻域解
def generate_neighborhood(v, deltas=[0.1, 0.03, 0.01]):
    neighbors = [v + delta for delta in deltas]
    return neighbors

# 爬山算法
def hill_climbing(v0, deltas, max_iterations=1000):
    current_solution = v0
    current_value = objective_function(current_solution)
    
    for iteration in range(max_iterations):
        # 生成邻域解
        neighbors = generate_neighborhood(current_solution, deltas)
        
        # 筛选出符合条件的邻域解（E(v_n') <= 2）
        valid_neighbors = []
        for neighbor in neighbors:
            neighbor_value = objective_function(neighbor)
            if neighbor_value <= 2:
                valid_neighbors.append((neighbor, neighbor_value))
        
        # 如果没有有效邻域解，停止
        if not valid_neighbors:
            break
        
        # 选择最优邻域解
        best_neighbor, best_value = max(valid_neighbors, key=lambda x: x[1])
        
        # 如果最优解不比当前解好，停止
        if best_value <= current_value:
            break
        
        # 更新当前解为最优邻域解
        current_solution = best_neighbor
        current_value = best_value
        
        # 输出每次迭代的信息
        print(f"Iteration {iteration}: v = {current_solution}, E(v) = {current_value}")
    
    return current_solution, current_value

# 执行算法
best_solution, best_value = hill_climbing(initial_solution, deltas=[0.1, 0.03, 0.01])

print(f"Best solution: v = {best_solution}, E(v) = {best_value}")
